# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
def clean_data(filename):
    if re.search('\[[\w ]+]',filename) is None:
        return filename
    else:
        return filename.replace(re.search('\[[\w ]*\]$', filename).group(), '')

In [3]:
def get_nhlarea(team):
    for each in list(city_nhlteams.index.values):
        if team in each:
            return city_nhlteams.at[each, 'Metropolitan area']
def get_nbaarea(team):
    for each in list(city_nbateams.index.values):
        if team in each:
            return city_nbateams.at[each, 'Metropolitan area']
def get_mlbarea(team):
    for each in list(city_mlbteams.index.values):
        if team in each:
            return city_mlbteams.at[each, 'Metropolitan area']
def get_nflarea(team):
    for each in list(city_nflteams.index.values):
        if team in each:
            return city_nflteams.at[each, 'Metropolitan area']

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year']==2018]
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'},inplace=True)

cities['NHL'] = cities['NHL'].apply(lambda x: clean_data(x))

city_nhlteams = cities[['Metropolitan area', 'NHL']].set_index('NHL')
city_nhlteams.drop(['—', ''], axis=0,inplace=True)

   
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_nhlarea(x))

out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)

population = cities[['Metropolitan area', 'Population']]
population = population.set_index('Metropolitan area')

new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
nhl = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
nhl['Population'] = pd.to_numeric(nhl['Population'])


def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nhl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
   
nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df[nba_df['year']==2018]
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities.rename(columns={'Population (2016 est.)[8]': 'Population'},inplace=True)
cities['NBA'] = cities['NBA'].apply(lambda x: clean_data(x))



city_nbateams = cities[['Metropolitan area', 'NBA']].set_index('NBA')
city_nbateams.drop(['—', ''], axis=0,inplace=True)


nba_df['team'].replace(to_replace = '(\*.\(\d+\))|\(\d+\)',value='',regex=True,inplace=True)
nba_df['team'] = nba_df['team'].apply(lambda x: x.strip())

nba_df['area'] = nba_df['team'].apply(lambda x: x.split()[-1])
nba_df['area'] = nba_df['area'].apply(lambda x: get_nbaarea(x))



out2 = []
for group, frame in nba_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out2_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out2.append(out2_dict)

population2 = cities[['Metropolitan area', 'Population']]
population2 = population2.set_index('Metropolitan area')

new2_df = pd.DataFrame(out2)
new2_df = new2_df.set_index('Area')
nba = pd.merge(new2_df, population2, how="inner", left_index=True, right_index=True)
nba['Population'] = pd.to_numeric(nba['Population'])

def nba_correlation():
    # YOUR CODE HERE
    
    
    population_by_region = nba['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nba['Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[mlb_df['year']==2018]

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'},inplace=True)
cities['MLB'] = cities['MLB'].apply(lambda x: clean_data(x))

city_mlbteams = cities[['Metropolitan area', 'MLB']].set_index('MLB')
city_mlbteams.drop(['—', ''], axis=0,inplace=True)

mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip())
mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
mlb_df['area'] = mlb_df['area'].apply(lambda x: get_mlbarea(x))
mlb_df.at[0, 'area'] = 'Boston'

out3 = []
for group, frame in mlb_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    mlb_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out3.append(mlb_dict)

population3 = cities[['Metropolitan area', 'Population']]
population3 = population3.set_index('Metropolitan area')

df3 = pd.DataFrame(out3)
df3 = df3.set_index('Area')
mlb = pd.merge(df3, population3, how="inner", left_index=True, right_index=True)
mlb['Population'] = pd.to_numeric(mlb['Population'])

def mlb_correlation(): 
    # YOUR CODE HERE
  
    
    population_by_region = mlb['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb['Ratio']# pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
  
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df['year']==2018]
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'},inplace=True)
cities['NFL'] = cities['NFL'].apply(lambda x: clean_data(x))

city_nflteams = cities[['Metropolitan area', 'NFL']].set_index('NFL')
city_nflteams.drop(['—', '', '— '], axis=0,inplace=True)

nfl_df['team'] = nfl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith(("*","+")) else x.strip())
nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])
nfl_df['area'] = nfl_df['area'].apply(lambda x: get_nflarea(x))
nfl_df = nfl_df[nfl_df['area'] != None]

out4 = []
for group, frame in nfl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out4_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out4.append(out4_dict)

population4 = cities[['Metropolitan area', 'Population']]
population4 = population4.set_index('Metropolitan area')

df4 = pd.DataFrame(out4)
df4 = df4.set_index('Area')
nfl= pd.merge(df4, population4, how="inner", left_index=True, right_index=True)
nfl['Population'] = pd.to_numeric(nfl['Population'])
def nfl_correlation(): 
    # YOUR CODE HERE
   
    
    population_by_region = nfl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl['Ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
nfl_correlation()

0.004922112149349407

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


NBA = nba.reset_index().rename(columns={'index':'State'}).set_index('State').drop('Population',axis=1)
NFL = nfl.reset_index().rename(columns={'index':'State'}).set_index('State').drop('Population',axis=1)
MLB = mlb.reset_index().rename(columns={'index':'State'}).set_index('State').drop('Population',axis=1)
NHL = nhl.reset_index().rename(columns={'index':'State'}).set_index('State').drop('Population',axis=1)


def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    sports = {"NFL": NFL, "NBA": NBA, "NHL": NHL, "MLB": MLB}
    def calculate_p_values(sports):
        
        p_values = pd.DataFrame(columns=sports.keys(), index=sports.keys())
        for (sports_name1, sports_df1) in sports.items():
            for (sports_name2, sports_df2) in sports.items():
                df_merge = pd.merge(sports_df1, sports_df2, on="State")
                p_values.loc[sports_name1, sports_name2] = stats.ttest_rel(df_merge["Ratio_x"],df_merge["Ratio_y"])[1]

        return p_values
    
    p_values = calculate_p_values(sports)
    p_values  = p_values.astype(np.float)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
